## **RQ1**

> *Do N-HiTS and N-BEATS maintain competitive performance across different atmospheric pollutants (PM2.5, PM10, O₃, NO₂) and forecast horizons?*
>> *Do global neural time series models (N-HiTS and N-BEATS) maintain competitive and robust performance across different atmospheric pollutants (PM2.5, PM10, O₃, NO₂) and forecast horizons (1–30 days), when compared to naïve, classical statistical, and simple global baselines, under both average and extreme-oriented evaluation metrics?*

* Evaluation across multiple horizons:

  * short-term (1–7 days),
  * medium-term (14 and 30 days).
* Comparison against naïve, statistical, and simple global forecasting baselines.
* Use of both average performance metrics and metrics explicitly oriented toward extreme events.

## Libraries

In [1]:
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'
import glob

import optuna
import itertools
import shutil
import time
import functools
import gc

import pandas as pd
import numpy as np
np.random.seed(1)
import scipy.stats as stats

import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots
import plotly.io as pio
from graphmodex import plotlymodex
pio.renderers.default = 'notebook'
from plotly.subplots import make_subplots

import joblib
import pickle
from IPython.display import clear_output

In [2]:
import neuralforecast
import mlforecast
import statsforecast
import utilsforecast
import coreforecast

from statsforecast import StatsForecast
from statsforecast.models import (
    Naive, SeasonalNaive, 
    AutoARIMA, AutoCES, AutoETS, AutoTheta,
)

from mlforecast import MLForecast
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from mlforecast.target_transforms import Differences
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

from neuralforecast import NeuralForecast
from neuralforecast.models import (
    NBEATS, NHITS,
    GRU, Informer, LSTM
)
from neuralforecast.losses.pytorch import MSE, SMAPE, MAE

from mlforecast.utils import PredictionIntervals

from pytorch_lightning import Trainer
trainer = Trainer(
    max_steps=4,
    logger=False,
    enable_progress_bar=False,
    enable_model_summary=False
)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")


# ==================================================
# REPRODUCTIBILITY
# ==================================================
import random
import torch

SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.


In [3]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="overflow encountered in square",
    category=RuntimeWarning,
)

In [4]:
def print_gpu_utilization():
    if torch.cuda.is_available():
        usage = torch.cuda.memory_reserved() / 1024**3 # GB
        total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"GPU usage: {usage:.2f}GB / {total:.2f}GB")

# Call this at the start of your loop

### Results Storage

In [5]:
from pathlib import Path

BASE_RESULTS = Path("Results/RQ1")
BASE_RESULTS.mkdir(parents=True, exist_ok=True)

def save_results(df, model_family, pollutant, horizon_label):
    out_dir = BASE_RESULTS / model_family
    out_dir.mkdir(parents=True, exist_ok=True)

    fname = f"{pollutant}_{horizon_label}.csv"
    df.to_csv(out_dir / fname, index=False)

BASE_DIR_Pilot = Path("Results/RQ1/Pilot")
(BASE_DIR_Pilot / "studies").mkdir(parents=True, exist_ok=True)
(BASE_DIR_Pilot / "tables").mkdir(parents=True, exist_ok=True)
(BASE_DIR_Pilot / "meta").mkdir(parents=True, exist_ok=True)

## Data & Ablation

In [6]:
# ===============================
# DATA
# ===============================
df = pd.read_parquet(r'..\Data\CAMS\processed\eac4_era5_2010_2024_brasil_enhanced.parquet')

pm10 = (
    df
    .rename(columns={
        'pm10': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

pm2p5 = (
    df
    .rename(columns={
        'pm2p5': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

go3 = (
    df
    .rename(columns={
        'go3': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

no2 = (
    df
    .rename(columns={
        'no2': 'y',
        'valid_time': 'ds'        
    })
    .query('state == "Paraná"')
    [['unique_id', 'ds', 'y']]
)

In [7]:
# go3.to_parquet(r'.\Results\RQ1\full\go3.parquet')
# pm10.to_parquet(r'.\Results\RQ1\full\nopm102.parquet')
# pm2p5.to_parquet(r'.\Results\RQ1\full\nopm2p52.parquet')
# no2.to_parquet(r'.\Results\RQ1\full\no2.parquet')

### Setup

In [8]:
# ===============================
# PILOT CONFIG
# ===============================
POLLUTANT_abl = 'pm10'
UID_abl = 0

steps_per_day = 8
horizons = {
    '1d': 1 * steps_per_day,
    '7d': 7 * steps_per_day,
    '14d': 14 * steps_per_day,
    '30d': 30 * steps_per_day,
}

# ===============================
# DATA
# ===============================
df_abl = (
    df
    .query("unique_id == @UID_abl")
    .rename(columns={'pm10': 'y', 'valid_time': 'ds'})
    [['unique_id', 'ds', 'y']]
    .sort_values('ds')
)

In [9]:
# ===============================
# CONFIG
# ===============================
steps_per_day = 8
two_year_steps = 2 * 365 * steps_per_day
target_windows = 30

FREQ = '3h'
SEASON_LENGTH = 8 

pollutants_dict = {
        'go3': {
            'df': go3,
            'scaler': 1e8
        },
        'no2': {
            'df': no2,
            'scaler': 1e10
        },
        'pm10': {
            'df': pm10,
            'scaler': 1e9
        },
        'pm2p5': {
            'df': pm2p5,
            'scaler': 1e9
        },
    }
experiments_dict = {
    '1 days': {
        'horizon': 8*1,
        'step_size': max(8*1, two_year_steps // target_windows),
        'windows': target_windows,
    },
    '7 days': {
        'horizon': 8*7,
        'step_size': max(8*7, two_year_steps // target_windows),
        'windows': target_windows,
    },
    '14 days': {
        'horizon': 8*14,
        'step_size': max(8*14, two_year_steps // target_windows),
        'windows': target_windows,
    },
    '30 days': {
        'horizon': 8*30,
        'step_size': 8*30,
        'windows': two_year_steps // (8*30),  # 24
    },
}

## **Models**

### Pilot for Ablation

In [ ]:
# =========================================================
# PILOT STUDY — MULTI-OBJECTIVE (MAE + MAE_p95)
# NHITS | 1 cell | 1 pollutant | multiple horizons
# =========================================================

# ---------------------------------------------------------
# HARDENING (CPU / Windows stability)
# ---------------------------------------------------------
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)

# ---------------------------------------------------------
# REPRODUCIBILITY
# ---------------------------------------------------------
SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ---------------------------------------------------------
# ASSUMED EXISTING OBJECTS
# ---------------------------------------------------------
# df_abl   : DataFrame with columns ['unique_id', 'ds', 'y']
#            containing ONLY ONE unique_id
# horizons : dict, e.g. {'1d':8,'7d':56,'14d':112,'30d':240}
# FREQ     : '3h'
# ---------------------------------------------------------

# ---------------------------------------------------------
# SAFE MAE_p95
# ---------------------------------------------------------
def mae_p95_safe(
    y: np.ndarray,
    yhat: np.ndarray,
    min_points: int = 10
) -> float:
    p95 = np.percentile(y, 95)
    mask = y >= p95
    if mask.sum() < min_points:
        return np.inf
    return float(np.mean(np.abs(y[mask] - yhat[mask])))

# ---------------------------------------------------------
# OPTUNA OBJECTIVE (MULTI-OBJECTIVE)
# ---------------------------------------------------------
def objective(trial, h: int):

    input_multiplier = trial.suggest_categorical(
        "input_multiplier", [2, 4, 6]
    )
    input_size = int(input_multiplier * h)

    model = NHITS(
        h=h,
        input_size=input_size,
        n_blocks=[1, 1, 1],
        mlp_units=3 * [[256, 256]],
        n_pool_kernel_size=[2, 2, 1],
        n_freq_downsample=[4, 2, 1],
        activation="ReLU",
        dropout_prob_theta=0.1,

        # Pilot setup
        max_steps=200,
        learning_rate=1e-3,
        batch_size=32,
        windows_batch_size=64,
        random_seed=SEED,
        loss=MAE(),
        logger=False,
    )

    nf = NeuralForecast(models=[model], freq=FREQ)

    cv = nf.cross_validation(
        df=df_abl,
        h=h,
        n_windows=10,
        step_size=h,
        refit=True,
    )

    y = cv["y"].to_numpy()
    yhat = cv["NHITS"].to_numpy()

    mae = float(np.mean(np.abs(y - yhat)))
    mae_p95 = mae_p95_safe(y, yhat, min_points=10)

    del nf, model, cv
    gc.collect()

    # MULTI-OBJECTIVE RETURN
    return mae, mae_p95

# ---------------------------------------------------------
# RUN PILOT
# ---------------------------------------------------------
pilot_records = []
pareto_summary = []

for label, h in horizons.items():

    print(f"\n🔎 Horizon: {label} ({h} steps)")

    sampler = optuna.samplers.TPESampler(seed=SEED)
    study = optuna.create_study(
        directions=["minimize", "minimize"],
        sampler=sampler,
    )

    study.optimize(
        lambda t: objective(t, h),
        n_trials=20,
        gc_after_trial=True,
    )

    study_path = BASE_DIR_Pilot / "studies" / f"study_horizon_{label}.pkl"

    with open(study_path, "wb") as f:
        pickle.dump(study, f)

    # Store all trials
    for t in study.trials:
        pilot_records.append({
            "horizon": label,
            "h": h,
            "input_multiplier": t.params.get("input_multiplier"),
            "MAE": t.values[0],
            "MAE_p95": t.values[1],
            "state": str(t.state),
        })

    # Store Pareto front
    for t in study.best_trials:
        pareto_summary.append({
            "horizon": label,
            "h": h,
            "input_multiplier": t.params["input_multiplier"],
            "MAE": t.values[0],
            "MAE_p95": t.values[1],
        })

# ---------------------------------------------------------
# RESULTS
# ---------------------------------------------------------
pilot_df = pd.DataFrame(pilot_records)
pareto_df = pd.DataFrame(pareto_summary)

print("\n=== PARETO FRONT (PER HORIZON) ===")
print(pareto_df.sort_values(["horizon", "MAE_p95", "MAE"]))

pilot_df_path = BASE_DIR_Pilot / "tables" / "pilot_trials.pkl"
pareto_df_path = BASE_DIR_Pilot / "tables" / "pilot_pareto.pkl"

pilot_df.to_pickle(pilot_df_path)
pareto_df.to_pickle(pareto_df_path)

pilot_df.to_csv(BASE_DIR_Pilot / "tables" / "pilot_trials.csv", index=False)
pareto_df.to_csv(BASE_DIR_Pilot / "tables" / "pilot_pareto.csv", index=False)

meta = {
    "seed": SEED,
    "freq": FREQ,
    "horizons": horizons,
    "n_trials": 20,
    "model": "NHITS",
    "loss": "MAE + MAE_p95",
    "date": pd.Timestamp.now().isoformat(),
}

with open(BASE_DIR_Pilot / "meta" / "experiment_meta.pkl", "wb") as f:
    pickle.dump(meta, f)

### Refit

In [ ]:
# Configurações do Teste de Refit
TEST_POLLUTANT = list(pollutants_dict.keys())[0] # Pega o primeiro poluente
TEST_HORIZON_LABEL = list(experiments_dict.keys())[2] # Pega o primeiro horizonte
H = experiments_dict[TEST_HORIZON_LABEL]['horizon']
WINDOWS = 5 # Reduzido para o teste não demorar horas
STEP_SIZE = experiments_dict[TEST_HORIZON_LABEL]['step_size']

refit_comparison = []

# Loop Comparativo
for refit_mode in [True, False]:
    print(f"\n🚀 Testando com refit={refit_mode}...")
    
    # Modelo simplificado para o teste
    model = NHITS(
        h=H,
        input_size=14*8,
        max_steps=300,
        random_seed=SEED,
        loss=MAE()
    )
    
    nf = NeuralForecast(models=[model], freq=FREQ)
    df_test = pollutants_dict[TEST_POLLUTANT]['df'].copy()
    df_test['y'] = df_test['y']*1e8
    # Tracking de tempo
    start_t = time.time()
    
    cv_results = nf.cross_validation(
        df=df_test,
        h=H,
        n_windows=WINDOWS,
        step_size=STEP_SIZE,
        refit=refit_mode
    )
    cv_results['y'] = cv_results['y']*1e-8
    cv_results['NHITS'] = cv_results['NHITS']*1e-8
    
    end_t = time.time()
    duration = end_t - start_t
    
    # Cálculo do Erro Médio Global no CV
    mae_val = np.mean(np.abs(cv_results['y'] - cv_results['NHITS']))
    
    refit_comparison.append({
        'refit': refit_mode,
        'MAE': mae_val,
        'Time_Sec': duration,
        'Time_Per_Window': duration / WINDOWS
    })

# --- ANÁLISE DOS RESULTADOS ---
comparison_df = pd.DataFrame(refit_comparison)
comparison_df['MAE_Diff_Pct'] = comparison_df['MAE'].pct_change() * 100
comparison_df['Speedup_Factor'] = comparison_df.iloc[0]['Time_Sec'] / comparison_df['Time_Sec']

print("\n=== COMPARAÇÃO: REFIT TRUE vs FALSE ===")
print(comparison_df.to_string(index=False))

# Verificação lógica
mae_diff = abs(comparison_df.iloc[0]['MAE'] - comparison_df.iloc[1]['MAE'])
print(f"\n💡 Diferença absoluta no MAE: {mae_diff:.6f}")
if mae_diff < 0.01: # Threshold arbitrário, ajuste conforme sua escala
    print("✅ Conclusão: A diferença de erro é insignificante. refit=False é seguro.")
else:
    print("⚠️ Atenção: Há diferença sensível no erro. Verifique a estabilidade dos dados.")

In [37]:
def plot_refit_publication_ready(df_comp):
    # Configurações de estilo para Artigo A1
    font_style = dict(family="Times New Roman", size=14, color="black")
    
    # Criar subplots com eixos Y secundários
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # 1. Adicionar Barra (Execution Time)
    fig.add_trace(
        go.Bar(
            x=df_comp['refit'].map({True: 'Refit (True)', False: 'No Refit (False)'}),
            y=df_comp['Time_Sec'],
            name="Total Execution Time",
            marker_color='lightslategray',
            opacity=0.7,
            text=[f"{v:.1f}s" for v in df_comp['Time_Sec']],
            textposition='outside',
            textfont=dict(family="Times New Roman", size=12)
        ),
        secondary_y=False,
    )

    # 2. Adicionar Linha (MAE)
    fig.add_trace(
        go.Scatter(
            x=df_comp['refit'].map({True: 'Refit (True)', False: 'No Refit (False)'}),
            y=df_comp['MAE'],
            name="Mean Absolute Error (MAE)",
            mode='lines+markers+text',
            marker=dict(color='firebrick', size=10, symbol='diamond'),
            line=dict(color='firebrick', width=3),
            text=[f"{v:.2e}" for v in df_comp['MAE']],
            textposition="top center",
            textfont=dict(family="Times New Roman", size=12)
        ),
        secondary_y=True,
    )

    # Customização de Layout (Academic Standard)
    fig.update_layout(
        title={
            'text': "Impact of Refit Strategy on Computational Performance and Accuracy",
            'y':0.95, 'x':0.475, 'xanchor': 'center', 'yanchor': 'top'
        },
        font=font_style,
        plot_bgcolor='white',
        paper_bgcolor='white',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=0.91,
            xanchor="right",
            x=0.93,
            bordercolor="Black",
            borderwidth=1,
            font_size=12
        ),
        margin=dict(l=80, r=80, t=70, b=80),
        width=700,
        height=600
    )

    # Configuração dos Eixos
    fig.update_xaxes(
        title_text="Cross-Validation Strategy",
        showline=True, linewidth=2, linecolor='black', mirror=True,
        gridcolor='rgba(0,0,0,0.1)'
    )
    
    fig.update_yaxes(
        title_text="<b>Execution Time (s)</b>",
        secondary_y=False,
        showline=True, linewidth=2, linecolor='black',
        gridcolor='rgba(0,0,0,0.1)',
        range=[0, df_comp['Time_Sec'].max() * 1.3] # Espaço para o label
    )
    
    fig.update_yaxes(
        title_text="<b>Mean Absolute Error (MAE)</b>",
        secondary_y=True,
        showline=True, linewidth=2, linecolor='black',
        tickformat=".2e", # Notação científica para valores pequenos
        range=[df_comp['MAE'].min() * 0.8, df_comp['MAE'].max() * 1.2]
    )

    return fig

# Executar plot
plot_refit_publication_ready(comparison_df).show()
plot_refit_publication_ready(comparison_df).write_image(
    r"..\Masters 26\Results\RQ1\MAE_refit_test_14h.pdf",
    format="pdf",
    width=700,
    height=600,
    scale=3  # aumenta qualidade de renderização
)
print(f"""
=== COMPARAÇÃO: REFIT TRUE vs FALSE ===
 refit          MAE   Time_Sec  Time_Per_Window  MAE_Diff_Pct  Speedup_Factor
  True 1.158920e-08 646.707652       129.341530           NaN        1.000000
 False 1.196151e-08 129.322932        25.864586      3.212619        5.000719

💡 Diferença absoluta no MAE: 3.723164e-10
✅ Conclusão: A diferença de erro é insignificante. refit=False é seguro.
""")


=== COMPARAÇÃO: REFIT TRUE vs FALSE ===
 refit          MAE   Time_Sec  Time_Per_Window  MAE_Diff_Pct  Speedup_Factor
  True 1.158920e-08 646.707652       129.341530           NaN        1.000000
 False 1.196151e-08 129.322932        25.864586      3.212619        5.000719

💡 Diferença absoluta no MAE: 3.723164e-10
✅ Conclusão: A diferença de erro é insignificante. refit=False é seguro.



In [38]:
# Definindo o caminho e nome do arquivo
output_path = r"..\Masters 26\Results\RQ1\MAE_refit_test_14h.png"

# Gerar a figura
fig = plot_refit_publication_ready(comparison_df)

# Salvar em PNG com SCALE ALTO (Isso garante a nitidez no Google Docs)
fig.write_image(
    output_path,
    format="png",
    width=700,
    height=600,
    scale=4  # Aumentamos para 4x para garantir nitidez extrema (equivalente a ~4k)
)

print(f"Imagem salva em alta resolução: {output_path}")

Imagem salva em alta resolução: ..\Masters 26\Results\RQ1\MAE_refit_test_14h.png


### Input Size

In [ ]:
# =====================================================
# DIEBOLD-MARIANO TEST (robusto para múltiplos horizontes)
# =====================================================

def diebold_mariano_test(e1, e2, h=1, power=1):
    """
    e1, e2: arrays de erro (y - y_hat)
    h: horizonte
    power: 1 = MAE, 2 = MSE
    """

    d = np.abs(e1)**power - np.abs(e2)**power
    mean_d = np.mean(d)
    T = len(d)

    # HAC variance (Newey-West style simplificado)
    gamma = []
    for lag in range(1, min(h, T)):
        cov = np.cov(d[:-lag], d[lag:])[0, 1]
        gamma.append(cov)

    var_d = np.var(d) + 2 * np.sum(gamma)

    dm_stat = mean_d / np.sqrt(var_d / T)
    p_value = 2 * (1 - stats.norm.cdf(np.abs(dm_stat)))

    return dm_stat, p_value


# =====================================================
# CONFIGURAÇÕES
# =====================================================

INPUT_SIZES = {
    "7d": 7 * 8,
    "14d": 14 * 8,
    "21d": 21 * 8
}

HORIZONS_TO_TEST = ["1 days", "30 days"]

MODELS = {
    "GRU": GRU,
    "LSTM": LSTM,
    "NBEATS": NBEATS,
    "NHITS": NHITS,
    "Informer": Informer,
}

all_predictions = []

# =====================================================
# LOOP PRINCIPAL — GERAR PREVISÕES
# =====================================================

for pollutant_name, pollutant_ in pollutants_dict.items():

    if pollutant_name != 'pm10':
        continue

    for horizon_label, experiment_ in experiments_dict.items():

        if horizon_label not in HORIZONS_TO_TEST:
            continue

        for model_name, ModelClass in MODELS.items():

            for input_label, input_size in INPUT_SIZES.items():

                print(f"{model_name} | {horizon_label} | {input_label}")

                model = ModelClass(
                    h=experiment_['horizon'],
                    input_size=input_size,
                    max_steps=200,
                    learning_rate=1e-3,
                    batch_size=32,
                    windows_batch_size=512,
                    random_seed=SEED,
                    alias=model_name,
                    logger=False,
                    loss=MAE(),
                )

                nf = NeuralForecast(models=[model], freq=FREQ)

                df_ = pollutant_['df'].copy()
                df_.y = df_.y * pollutant_['scaler']

                results_nf = nf.cross_validation(
                    df=df_,
                    h=experiment_['horizon'],
                    n_windows=experiment_['windows'],
                    step_size=experiment_['step_size'],
                    refit=False,
                )

                # scale back
                scale = pollutant_['scaler']
                results_nf['y'] /= scale
                results_nf[model_name] /= scale

                results_nf['model'] = model_name
                results_nf['input_label'] = input_label
                results_nf['horizon'] = horizon_label

                all_predictions.append(results_nf)

# Consolidar todas previsões
pred_df = pd.concat(all_predictions).reset_index(drop=True)


# =====================================================
# DIEBOLD-MARIANO ENTRE INPUT_SIZES
# =====================================================

dm_results = []

for model_name in MODELS.keys():

    for horizon_label in HORIZONS_TO_TEST:

        subset = pred_df[
            (pred_df['model'] == model_name) &
            (pred_df['horizon'] == horizon_label)
        ]

        horizon_value = experiments_dict[horizon_label]['horizon']

        input_labels = list(INPUT_SIZES.keys())

        for i in range(len(input_labels)):
            for j in range(i + 1, len(input_labels)):

                input_a = input_labels[i]
                input_b = input_labels[j]

                df_a = subset[subset['input_label'] == input_a]
                df_b = subset[subset['input_label'] == input_b]

                e1 = df_a['y'].values - df_a[model_name].values
                e2 = df_b['y'].values - df_b[model_name].values

                dm_stat, p_value = diebold_mariano_test(
                    e1, e2,
                    h=horizon_value,
                    power=1
                )

                dm_results.append({
                    "model": model_name,
                    "horizon": horizon_label,
                    "input_a": input_a,
                    "input_b": input_b,
                    "dm_stat": dm_stat,
                    "p_value": p_value
                })

dm_df = pd.DataFrame(dm_results)

print("\nDiebold-Mariano between input_sizes:")
display(dm_df.sort_values(["model", "horizon", "input_a"]))


### Epochs

In [ ]:
from pytorch_lightning.loggers import CSVLogger

# --- Configurações de Dados ---
pollutant_name = list(pollutants_dict.keys())[0]
pollutant_ = pollutants_dict[pollutant_name]
horizon = 56
input_size = 56
FREQ = '3H' 
SEED = 42

df_ = pollutant_['df'].copy()
df_.y = df_.y * pollutant_['scaler']
cutoff = df_['ds'].max() - pd.Timedelta(days=30)
train_df = df_[df_['ds'] <= cutoff]

# --- Definição dos Modelos ---
# Criamos uma lista de dicionários para iterar e criar loggers automaticamente
model_configs = [
    (Informer, 'Informer'),
    (GRU, 'GRU'),
    (LSTM, 'LSTM'),
    (NHITS, 'NHITS'),
    (NBEATS, 'NBEATS'),
]

models = []
for model_class, name in model_configs:
    logger = CSVLogger(r"Results/RQ1/logs_science", name=name)
    
    # Instanciação dinâmica
    m = model_class(
        h=horizon,
        input_size=input_size,
        max_steps=1500, # Aumentado conforme seu snippet
        learning_rate=1e-3,
        batch_size=32,
        windows_batch_size=1024,
        random_seed=SEED,
        alias=name,
        loss=MAE(),
        logger=logger,
    )
    models.append(m)

# Inicialização do NeuralForecast
nf = NeuralForecast(models=models, freq=FREQ)

# Treino
nf.fit(df=train_df, val_size=horizon)

print("Treinamento de todos os modelos finalizado!")

In [ ]:
def get_latest_metrics(model_alias):
    log_path = f"Results/RQ1/logs_science/{model_alias}/"
    versions = sorted(glob.glob(os.path.join(log_path, "version_*")), key=os.path.getmtime)
    if not versions:
        return None
    csv_path = os.path.join(versions[-1], "metrics.csv")
    return pd.read_csv(csv_path).groupby('step').mean().reset_index()


colors = {
    'NHITS': "#104266",
    'NBEATS': "#c49962",
    'Informer': "#6db16d",
    'LSTM': "#bb69b7",
    'GRU': "#9c3e3e"
}

fig = go.Figure()

for _, name in model_configs:
    metrics = get_latest_metrics(name)
    if metrics is not None:
        fig.add_trace(go.Scatter(
            x=metrics['step'],
            y=metrics['train_loss_epoch'],
            mode='lines',
            name=name,
            line=dict(color=colors[name], width=2),
            legendgroup=name
        ))

# =============================
# Layout Acadêmico Refinado
# =============================

fig.update_layout(
    template="simple_white",
    title={
        'text': "Learning Curves Comparison: Ozone",
        'font': {'family': "Times New Roman", 'size': 24},
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis=dict(
        title=dict(text="Training Steps", font=dict(family="Times New Roman", size=20)),
        tickfont=dict(family="Times New Roman", size=16),
        showline=True,
        linewidth=1.2,
        linecolor='black',
        mirror=True,
        showgrid=True,
        gridcolor='rgba(0,0,0,0.12)',  # grid MUITO leve
        zeroline=False
    ),
    yaxis=dict(
        title=dict(text="Mean Absolute Error (MAE)", font=dict(family="Times New Roman", size=20)),
        tickfont=dict(family="Times New Roman", size=16),
        showline=True,
        linewidth=1.2,
        linecolor='black',
        mirror=True,
        showgrid=True,
        gridcolor='rgba(0,0,0,0.12)',  # grid MUITO leve
        zeroline=False
    ),
    legend=dict(
        font=dict(family="Times New Roman", size=14),
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.48
    ),
    width=900,
    height=600,
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(l=80, r=40, t=100, b=80)
)

fig.show()
fig.write_image(
    r"..\Masters 26\Results\RQ1\Learning_Curves.pdf",
    format="pdf",
    width=700,
    height=600,
    scale=3  # aumenta qualidade de renderização
)

### Deep Learners

In [ ]:
for pollutant_name, pollutant_ in pollutants_dict.items():
    for horizon_label, experiment_ in experiments_dict.items():
        try:
            print_gpu_utilization()
            clear_output(wait=True)
            print(f"Running NEURAL | {pollutant_name} | {horizon_label}")

            # -------------------------------
            # MODELS
            # -------------------------------
            models = [
                GRU(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    max_steps=1000,  # Passos máximos
                    learning_rate=1e-3,  # Taxa de aprendizado
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='GRU',  # Alias para identificação
                    loss=MAE(),  # Função de perda
                    logger=False,
                ),
                Informer(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    max_steps=1500,  # Passos máximos
                    learning_rate=1e-3,  # Taxa de aprendizado
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='Informer',
                    loss=MAE(),  # Função de perda
                    logger=False,
                ),
                LSTM(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    max_steps=1000,  # Passos máximos
                    learning_rate=1e-3,  # Taxa de aprendizado
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='LSTM',  # Alias para identificação
                    loss=MAE(),  # Função de perda
                    logger=False,
                ),
            ]

            nf = NeuralForecast(models=models, freq=FREQ)

            # -------------------------------
            # DATA
            # -------------------------------
            df_ = pollutant_['df'].copy()
            df_.y = df_.y * pollutant_['scaler']

            start_time = time.time()
            results_nf = nf.cross_validation(
                df=df_,
                h=experiment_['horizon'],
                n_windows=experiment_['windows'],
                step_size=experiment_['step_size'],
                refit=False,
            )
            end_time = time.time()
            
            total_time = end_time - start_time
            time_per_window = total_time / experiment_['windows']
            results_nf['fit_time_seconds'] = total_time

            # -------------------------------
            # SCALE BACK
            # -------------------------------
            scale = pollutant_['scaler']
            for col in ['y', 'GRU', 'Informer', 'LSTM']:
                results_nf[col] = results_nf[col] / scale

            # -------------------------------
            # SAVE
            # -------------------------------
            save_results(
                results_nf,
                model_family="dl",
                pollutant=pollutant_name,
                horizon_label=horizon_label.replace(" ", "")
            )

            del nf
            del models
            gc.collect()
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"Failed on {pollutant_name} {horizon_label}: {e}")
            continue # Move to next instead of crashing the whole notebook

### Neural Forecasters

In [ ]:
for pollutant_name, pollutant_ in pollutants_dict.items():
    for horizon_label, experiment_ in experiments_dict.items():
        try:
            print_gpu_utilization()
            clear_output(wait=True)
            print(f"Running NEURAL | {pollutant_name} | {horizon_label}")

            # -------------------------------
            # MODELS
            # -------------------------------
            models = [
                NBEATS(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    stack_types=["identity", "trend", "seasonality"],
                    n_blocks=[1, 1, 1],
                    mlp_units=3 * [[256, 256]],
                    basis='polynomial',
                    n_basis=2,
                    n_harmonics=2,
                    shared_weights=True,
                    activation='ReLU',
                    max_steps=1000,
                    learning_rate=1e-3,
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='NBEATS-I',
                    loss=MAE(),
                ),
                NBEATS(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    stack_types=['identity'] * 3,
                    n_blocks=[2, 2, 2],
                    mlp_units=3 * [[256, 256]],
                    shared_weights=False,
                    activation='ReLU',
                    max_steps=1000,
                    learning_rate=1e-3,
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='NBEATS-G',
                    logger=False,
                    loss=MAE(),
                ),
                NHITS(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    n_blocks=[1, 1, 1],
                    mlp_units=3 * [[256, 256]],
                    n_pool_kernel_size=[2, 2, 1],
                    n_freq_downsample=[4, 2, 1],
                    activation='ReLU',
                    dropout_prob_theta=0.1,
                    max_steps=1000,
                    learning_rate=1e-3,
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='NHITS',
                    logger=False,
                    loss=MAE(),
                ),
            ]

            nf = NeuralForecast(models=models, freq=FREQ)

            # -------------------------------
            # DATA
            # -------------------------------
            df_ = pollutant_['df'].copy()
            df_.y = df_.y * pollutant_['scaler']

            start_time = time.time()
            results_nf = nf.cross_validation(
                df=df_,
                h=experiment_['horizon'],
                n_windows=experiment_['windows'],
                step_size=experiment_['step_size'],
                refit=False,
            )
            end_time = time.time()
            
            total_time = end_time - start_time
            time_per_window = total_time / experiment_['windows']
            results_nf['fit_time_seconds'] = total_time

            # -------------------------------
            # SCALE BACK
            # -------------------------------
            scale = pollutant_['scaler']
            for col in ['y', 'NHITS', 'NBEATS-G', 'NBEATS-I']:
                results_nf[col] = results_nf[col] / scale

            # -------------------------------
            # SAVE
            # -------------------------------
            save_results(
                results_nf,
                model_family="neural",
                pollutant=pollutant_name,
                horizon_label=horizon_label.replace(" ", "")
            )

            del nf
            del models
            gc.collect()
            torch.cuda.empty_cache()
            
        except Exception as e:
            print(f"Failed on {pollutant_name} {horizon_label}: {e}")
            continue # Move to next instead of crashing the whole notebook

### Statistical Forecasters

In [ ]:
for pollutant_name, pollutant_ in pollutants_dict.items():
    for horizon_label, experiment_ in experiments_dict.items():

        print(f"Running STATS | {pollutant_name} | {horizon_label}")

        # -------------------------------
        # MODELS
        # -------------------------------
        stat_models = [
            AutoARIMA(season_length=SEASON_LENGTH, alias='Arima'),
            AutoETS(season_length=SEASON_LENGTH, alias='ETS'),
            AutoTheta(season_length=SEASON_LENGTH, alias='Theta'),
            SeasonalNaive(season_length=SEASON_LENGTH, alias='SeasonalNaive-8'),
            SeasonalNaive(season_length=56, alias='SeasonalNaive-56'),
            Naive(alias='Naive'),
        ]

        sf = StatsForecast(
            models=stat_models,
            freq=FREQ,
            n_jobs=1,
        )

        # -------------------------------
        # DATA
        # -------------------------------
        df_ = pollutant_['df'].copy()

        # -------------------------------
        # CROSS-VALIDATION
        # -------------------------------
        start_time = time.time()
        results_sf = sf.cross_validation(
            df=df_,
            h=experiment_['horizon'],
            n_windows=experiment_['windows'],
            step_size=experiment_['step_size'],
            input_size=14*8,
            refit=True,
        )
        end_time = time.time()
        
        total_time = end_time - start_time
        time_per_window = total_time / experiment_['windows']
        results_sf['fit_time_seconds'] = total_time
        
        # -------------------------------
        # SAVE
        # -------------------------------
        save_results(
            results_sf,
            model_family="stats",
            pollutant=pollutant_name,
            horizon_label=horizon_label.replace(" ", "")
        )

### Machine Learners

In [ ]:
for pollutant_name, pollutant_ in pollutants_dict.items():

    for horizon_label, experiment_ in experiments_dict.items():

        try:
            clear_output(wait=True)
            print(f"Running ML | {pollutant_name} | {horizon_label}")

            # -------------------------------
            # MODEL 
            # -------------------------------
            lgb_model = lgb.LGBMRegressor(
                n_estimators=500,
                learning_rate=0.05,
                num_leaves=31,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=SEED,
                n_jobs=1,
            )
            rf_model = RandomForestRegressor(
                n_estimators=500,
                max_depth=10,
                random_state=SEED,
                n_jobs=1,
            )
            xgb_model = xgb.XGBRegressor(
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=SEED,
                n_jobs=1,
            )

            # -------------------------------
            # MLForecast (lags + calendar)
            # -------------------------------
            mlf = MLForecast(
                models={
                    'RandomForest': rf_model,
                    'XGBoost': xgb_model,
                    'LightGBM': lgb_model,
                },
                freq=FREQ,
                lags=[1, 2, 4, 8, 16, 24, 56, 112],
                lag_transforms={
                    8:  [RollingMean(window_size=8), ExpandingMean()],
                    56: [RollingMean(window_size=56)],
                },
                date_features=[
                    'hour',       # ciclo intradiário (3h)
                    'dayofweek',  # ciclo semanal
                    'month',      # sazonalidade anual
                    'dayofyear',  # sazonalidade anual mais “contínua”
                ],
            )

            # -------------------------------
            # DATA
            # -------------------------------
            df_ = pollutant_['df'].copy()
            df_ = df_.sort_values(['unique_id', 'ds'])

            # (opcional, mas recomendado) garantir dtype datetime
            df_['ds'] = pd.to_datetime(df_['ds'])

            # -------------------------------
            # CROSS-VALIDATION
            # -------------------------------
            start_time = time.time()
            results_ml = mlf.cross_validation(
                df=df_,
                h=experiment_['horizon'],
                n_windows=experiment_['windows'],
                step_size=experiment_['step_size'],
                refit=False,
            )
            end_time = time.time()

            total_time = end_time - start_time
            results_ml['fit_time_seconds'] = total_time

            # -------------------------------
            # SAVE
            # -------------------------------
            save_results(
                results_ml,
                model_family="ml",
                pollutant=pollutant_name,
                horizon_label=horizon_label.replace(" ", "")
            )

            del mlf
            del rf_model, xgb_model, lgb_model
            
        except Exception as e:
            print(f"Failed on {pollutant_name} {horizon_label}: {e}")
            continue # Move to next instead of crashing the whole notebook

## Merge

In [13]:
FULL_DIR = BASE_RESULTS / "full"
FULL_DIR.mkdir(parents=True, exist_ok=True)

def build_full_results(pollutant, horizon_label):
    """
    Concatena neural + stats + ml para um (pollutant, horizon)
    """
    dfs = []

    for family in ["neural", "stats", "ml", "dl"]:
        fpath = BASE_RESULTS / family / f"{pollutant}_{horizon_label}.csv"

        if fpath.exists():
            df = pd.read_csv(fpath)

            # Remove a coluna 'fit_time' se ela existir, pois ela não deve ser parte do merge
            if 'fit_time_seconds' in df.columns:
                df = df.drop(columns=['fit_time_seconds'])
            
            dfs.append(df)
        else:
            print(f"⚠️ Missing: {fpath}")

    if not dfs:
        raise ValueError("No result files found.")

    # Merge progressivo (chaves comuns)
    df_full = dfs[0]
    for df in dfs[1:]:
        df_full = df_full.merge(
            df,
            on=["unique_id", "ds", "cutoff", "y"],
            how="inner"
        )

    return df_full

for pollutant_name in pollutants_dict.keys():
    for horizon_label in experiments_dict.keys():

        horizon_clean = horizon_label.replace(" ", "")

        print(f"Building FULL | {pollutant_name} | {horizon_clean}")

        df_full = build_full_results(
            pollutant=pollutant_name,
            horizon_label=horizon_clean
        )

        df_full.to_csv(
            FULL_DIR / f"{pollutant_name}_{horizon_clean}.csv",
            index=False
        )

Building FULL | go3 | 1days
Building FULL | go3 | 7days
Building FULL | go3 | 14days
Building FULL | go3 | 30days
Building FULL | no2 | 1days
Building FULL | no2 | 7days
Building FULL | no2 | 14days
Building FULL | no2 | 30days
Building FULL | pm10 | 1days
Building FULL | pm10 | 7days
Building FULL | pm10 | 14days
Building FULL | pm10 | 30days
Building FULL | pm2p5 | 1days
Building FULL | pm2p5 | 7days
Building FULL | pm2p5 | 14days
Building FULL | pm2p5 | 30days


## **Estatísticas**

In [ ]:
# ==================================================
# METRICS
# ==================================================

def mae(y, yhat):
    return np.mean(np.abs(y - yhat))


def mse(y, yhat):
    return np.mean((y - yhat) ** 2)


def rmse(y, yhat):
    return np.sqrt(mse(y, yhat))


def smape(y, yhat):
    denom = (np.abs(y) + np.abs(yhat)) / 2
    mask = denom != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs(y[mask] - yhat[mask]) / denom[mask])


def mae_conditional(y, yhat, threshold):
    mask = y >= threshold
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs(y[mask] - yhat[mask]))


def bias_conditional(y, yhat, threshold):
    mask = y >= threshold
    if mask.sum() == 0:
        return np.nan
    return np.mean(yhat[mask] - y[mask])


def skill_score(model_err, baseline_err):
    if baseline_err == 0 or np.isnan(baseline_err):
        return np.nan
    return 1 - model_err / baseline_err


def extreme_event_metrics(y, yhat, threshold):
    y_event = y >= threshold
    yhat_event = yhat >= threshold

    tp = np.sum(y_event & yhat_event)
    fp = np.sum(~y_event & yhat_event)
    fn = np.sum(y_event & ~yhat_event)

    precision = tp / (tp + fp) if (tp + fp) > 0 else np.nan
    recall = tp / (tp + fn) if (tp + fn) > 0 else np.nan

    if precision > 0 and recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = np.nan

    return precision, recall, f1


# ==================================================
# EVALUATION LOOP (ROBUST & NO LEAKAGE)
# ==================================================

BASE = Path("Results/RQ1/full")
baseline_name = "Naive"

STEPS_PER_YEAR = 365 * 8  # dados 3h
WINDOW_P95 = STEPS_PER_YEAR

records = []

for file in BASE.glob("*.csv"):

    pollutant, horizon = file.stem.split("_", 1)
    print(f"Evaluating | {pollutant} | {horizon}")

    # --------------------------------------
    # LOAD PREDICTIONS
    # --------------------------------------
    df_pred = pd.read_csv(file)
    df_pred['ds'] = pd.to_datetime(df_pred['ds'])
    df_pred['cutoff'] = pd.to_datetime(df_pred['cutoff'])

    # --------------------------------------
    # LOAD FULL GROUND TRUTH
    # --------------------------------------
    df_true = pollutants_dict[pollutant]['df'].copy()
    df_true['ds'] = pd.to_datetime(df_true['ds'])

    # --------------------------------------
    # DEFINE MODEL COLUMNS (CLEAN)
    # --------------------------------------
    model_cols = [
        c for c in df_pred.columns
        if (
            c not in ["unique_id", "ds", "cutoff", "y"]
            and not c.startswith("fit_time")
            and df_pred[c].dtype in [np.float64, np.float32]
        )
    ]

    if baseline_name not in model_cols:
        raise ValueError(f"Baseline '{baseline_name}' not found.")

    # --------------------------------------
    # GROUP BY FOLD
    # --------------------------------------
    for (uid, cutoff), df_fold in df_pred.groupby(["unique_id", "cutoff"]):

        # ----------------------------------
        # TRAIN WINDOW (NO LEAKAGE)
        # ----------------------------------
        y_train_full = (
            df_true
            .query("unique_id == @uid and ds <= @cutoff")
            ['y']
            .values
        )

        if len(y_train_full) < WINDOW_P95:
            continue

        y_train_recent = y_train_full[-WINDOW_P95:]
        p95 = np.percentile(y_train_recent, 95)

        # ----------------------------------
        # TEST HORIZON
        # ----------------------------------
        y_full = df_fold['y'].values
        y_base_full = df_fold[baseline_name].values

        # baseline válido
        mask_base_valid = ~np.isnan(y_base_full)

        if mask_base_valid.sum() == 0:
            continue

        # ----------------------------------
        # EVALUATE EACH MODEL
        # ----------------------------------
        for model in model_cols:

            yhat_full = df_fold[model].values
            mask_valid = ~np.isnan(yhat_full)

            if mask_valid.sum() == 0:
                continue

            y_valid = y_full[mask_valid]
            yhat = yhat_full[mask_valid]
            y_base_valid = y_base_full[mask_valid]

            # métricas gerais do modelo
            mae_model = mae(y_valid, yhat)
            rmse_model = rmse(y_valid, yhat)
            smape_model = smape(y_valid, yhat)

            # métricas do baseline (ajustadas ao mesmo subconjunto)
            mae_base_model = mae(y_valid, y_base_valid)
            rmse_base_model = rmse(y_valid, y_base_valid)
            smape_base_model = smape(y_valid, y_base_valid)
            mae_base_model_p95 = mae_conditional(y_valid, y_base_valid, p95)

            # Skill Scores gerais
            skill_mae = skill_score(mae_model, mae_base_model)
            skill_rmse = skill_score(rmse_model, rmse_base_model)
            skill_smape = skill_score(smape_model, smape_base_model)

            # extremos
            mae_p95 = mae_conditional(y_valid, yhat, p95)
            skill_p95 = skill_score(mae_p95, mae_base_model_p95)
            bias_p95 = bias_conditional(y_valid, yhat, p95)

            precision, recall, f1 = extreme_event_metrics(
                y_valid, yhat, p95
            )

            records.append({
                "pollutant": pollutant,
                "horizon": horizon,
                "unique_id": uid,
                "cutoff": cutoff,
                "model": model,

                # métricas médias e seus respectivos skill scores
                "MAE": mae_model,
                "Skill_MAE": skill_mae,
                "RMSE": rmse_model,
                "Skill_RMSE": skill_rmse,
                "sMAPE": smape_model,
                "Skill_sMAPE": skill_smape,

                # extremos
                "MAE_p95": mae_p95,
                "Skill_p95": skill_p95,
                "Bias_p95": bias_p95,
                "Precision_p95": precision,
                "Recall_p95": recall,
                "F1_p95": f1,
            })

# ==================================================
# FINAL DATAFRAME
# ==================================================

metrics_df = pd.DataFrame(records)
metrics_df.to_csv(r'..\Masters 26\Results\RQ1\metrics.csv', index=False)

Evaluating | go3 | 14days
Evaluating | go3 | 1days
Evaluating | go3 | 30days
Evaluating | go3 | 7days
Evaluating | no2 | 14days
Evaluating | no2 | 1days
Evaluating | no2 | 30days
Evaluating | no2 | 7days
Evaluating | pm10 | 14days
Evaluating | pm10 | 1days
Evaluating | pm10 | 30days
Evaluating | pm10 | 7days
Evaluating | pm2p5 | 14days
Evaluating | pm2p5 | 1days
Evaluating | pm2p5 | 30days
Evaluating | pm2p5 | 7days


In [51]:
summary = (
    metrics_df
    .groupby(["model"])
    .agg({
        "MAE": "mean",
        "RMSE": "mean",
        "sMAPE": "mean",
        "MAE_p95": "mean",
        "Skill_p95": "mean",
        "F1_p95": "mean"
    })
    .sort_values("MAE")
)
summary

,MAE,RMSE,sMAPE,MAE_p95,Skill_p95,F1_p95
model,,,,,,
NHITS,5.482232e-09,7.119524e-09,0.433220,1.429769e-08,0.068600,0.527031
NBEATS-G,5.486953e-09,7.120524e-09,0.433298,1.428247e-08,0.088899,0.523155
NBEATS-I,5.502032e-09,7.150968e-09,0.436218,1.428918e-08,0.098030,0.516337
LightGBM,5.812858e-09,7.385411e-09,0.436081,1.421793e-08,0.077790,0.498773
LSTM,5.912126e-09,7.554950e-09,0.462718,1.465362e-08,-0.459748,0.540404
GRU,5.976189e-09,7.626346e-09,0.466126,1.490091e-08,-0.461215,0.551713
Informer,6.410303e-09,8.244619e-09,0.542345,1.994525e-08,-0.788818,0.556775
RandomForest,6.692651e-09,8.406920e-09,0.578508,1.775421e-08,-1.075025,0.681198
Theta,7.318015e-09,8.966578e-09,0.537794,1.528884e-08,0.053667,0.520314


# Computational Cost

In [ ]:
benchmark_all = []

for pollutant_name, pollutant_ in pollutants_dict.items():

    if pollutant_name != "go3":
        continue

    for horizon_label, experiment_ in experiments_dict.items():

        models = [
                NBEATS(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    stack_types=["identity", "trend", "seasonality"],
                    n_blocks=[1, 1, 1],
                    mlp_units=3 * [[256, 256]],
                    basis='polynomial',
                    n_basis=2,
                    n_harmonics=2,
                    shared_weights=True,
                    activation='ReLU',
                    max_steps=1000,
                    learning_rate=1e-3,
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='NBEATS-I',
                    loss=MAE(),
                ),
                NBEATS(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    stack_types=['identity'] * 3,
                    n_blocks=[2, 2, 2],
                    mlp_units=3 * [[256, 256]],
                    shared_weights=False,
                    activation='ReLU',
                    max_steps=1000,
                    learning_rate=1e-3,
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='NBEATS-G',
                    logger=False,
                    loss=MAE(),
                ),
                NHITS(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    n_blocks=[1, 1, 1],
                    mlp_units=3 * [[256, 256]],
                    n_pool_kernel_size=[2, 2, 1],
                    n_freq_downsample=[4, 2, 1],
                    activation='ReLU',
                    dropout_prob_theta=0.1,
                    max_steps=1000,
                    learning_rate=1e-3,
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='NHITS',
                    logger=False,
                    loss=MAE(),
                ),
                GRU(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    max_steps=1000, 
                    learning_rate=1e-3, 
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='GRU', 
                    loss=MAE(), 
                    logger=False,
                ),
                Informer(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    max_steps=1500, 
                    learning_rate=1e-3, 
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='Informer',
                    loss=MAE(), 
                    logger=False,
                ),
                LSTM(
                    h=experiment_['horizon'],
                    input_size=14*8,
                    max_steps=1000, 
                    learning_rate=1e-3, 
                    batch_size=32,
                    windows_batch_size=1024,
                    random_seed=SEED,
                    alias='LSTM', 
                    loss=MAE(), 
                    logger=False,
                ),
            ]

        for model_ in models:

            print(f'{horizon_label} + {model_}')

            nf = NeuralForecast(models=[model_], freq=FREQ)

            df_ = pollutant_['df'].copy()
            df_.y = df_.y * pollutant_['scaler']

            n_series = df_['unique_id'].nunique()

            start_time = time.time()
            nf.cross_validation(
                df_, 
                h=experiment_['horizon'], 
                n_windows=experiment_['windows'], 
                step_size=experiment_['step_size'], 
                refit=False
            )
            total_time = time.time() - start_time

            benchmark_all.append({
                "pollutant": pollutant_name,
                "horizon_label": horizon_label,
                "horizon_steps": experiment_['horizon'],
                "model_family": "neural",
                "model": model_.alias,
                "training_time_sec": total_time,
                "time_per_window_sec": total_time / experiment_['windows'],
                "time_per_series_sec": total_time / n_series,
                "n_series": n_series,
                "n_windows": experiment_['windows']
            })

            del nf
            torch.cuda.empty_cache()
            gc.collect()

            clear_output(wait=True)

In [16]:
df_benchmark = pd.DataFrame(benchmark_all)
df_benchmark.to_csv("computational_benchmark_neural.csv", index=False)

In [17]:
for pollutant_name, pollutant_ in pollutants_dict.items():

    if pollutant_name != "go3":
        continue

    for horizon_label, experiment_ in experiments_dict.items():

        lgb_model = lgb.LGBMRegressor(
            n_estimators=500,
            learning_rate=0.05,
            num_leaves=31,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=SEED,
            n_jobs=3-1,
        )
        rf_model = RandomForestRegressor(
            n_estimators=500,
            max_depth=10,
            random_state=SEED,
            n_jobs=-1,
        )
        xgb_model = xgb.XGBRegressor(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=SEED,
            n_jobs=-1,
        )

        models_dict = {
            'LightGBM': lgb_model,
            'RandomForest': rf_model,
            'XGBoost': xgb_model,
        }

        df_ = pollutant_['df'].copy()
        df_ = df_.sort_values(['unique_id', 'ds'])
        df_['ds'] = pd.to_datetime(df_['ds'])

        n_series = df_['unique_id'].nunique()

        for model_name, model_obj in models_dict.items():

            print(f'{horizon_label} + {model_name}')

            mlf = MLForecast(
                models={model_name: model_obj},
                freq=FREQ,
                lags=[1,2,4,8,16,24,56,112],
                lag_transforms={
                    8:[RollingMean(8), ExpandingMean()],
                    56:[RollingMean(56)]
                },
                date_features=['hour','dayofweek','month','dayofyear'],
            )

            start_time = time.time()

            mlf.cross_validation(
                df=df_,
                h=experiment_['horizon'],
                n_windows=experiment_['windows'],
                step_size=experiment_['step_size'],
                refit=False,
            )

            total_time = time.time() - start_time

            benchmark_all.append({
                "pollutant": pollutant_name,
                "horizon_label": horizon_label,
                "horizon_steps": experiment_['horizon'],
                "model_family": "ml",
                "model": model_name,
                "training_time_sec": total_time,
                "time_per_window_sec": total_time / experiment_['windows'],
                "time_per_series_sec": total_time / n_series,
                "n_series": n_series,
                "n_windows": experiment_['windows']
            })

            del mlf
            gc.collect()

            clear_output(wait=True)

30 days + XGBoost


In [18]:
df_benchmark = pd.DataFrame(benchmark_all)
df_benchmark.to_csv("computational_benchmark_neural_ml.csv", index=False)

In [13]:
benchmark_all = []

for pollutant_name, pollutant_ in pollutants_dict.items():

    if pollutant_name != "go3":
        continue

    for horizon_label, experiment_ in experiments_dict.items():

        stat_models = [
            AutoARIMA(season_length=SEASON_LENGTH, alias='Arima'),
            AutoETS(season_length=SEASON_LENGTH, alias='ETS'),
            AutoTheta(season_length=SEASON_LENGTH, alias='Theta'),
            SeasonalNaive(season_length=SEASON_LENGTH, alias='SeasonalNaive-8'),
            SeasonalNaive(season_length=56, alias='SeasonalNaive-56'),
            Naive(alias='Naive'),
        ]

        df_ = pollutant_['df'].copy()
        n_series = 3

        for model_ in stat_models:

            print(f'{horizon_label} + {model_.alias}')

            sf = StatsForecast(
                models=[model_],
                freq=FREQ,
                n_jobs=-1,
            )

            # -------------------
            # FIT TIME
            # -------------------
            start_fit = time.time()
            results_sf = sf.cross_validation(
                df=df_.query('unique_id.isin([54, 55, 56])'), 
                h=experiment_['horizon'], 
                n_windows=experiment_['windows'], 
                step_size=experiment_['step_size'], 
                refit=False,
            )
            total_time = time.time() - start_fit

            benchmark_all.append({
                "pollutant": pollutant_name,
                "horizon_label": horizon_label,
                "horizon_steps": experiment_['horizon'],
                "model_family": "stats",
                "model": model_.alias,
                "training_time_sec": total_time,
                "time_per_window_sec": total_time / experiment_['windows'],
                "time_per_series_sec": total_time / n_series,
                "n_series": n_series,
                "n_windows": experiment_['windows']
            })

            del sf
            gc.collect()

            clear_output(wait=True)

30 days + Naive


In [14]:
df_benchmark = pd.DataFrame(benchmark_all)
df_benchmark.to_csv("computational_benchmark_stats.csv", index=False)

In [25]:
df_benchmark = pd.read_csv("computational_benchmark_neural_ml.csv")
df_benchmark = pd.concat([df_benchmark, pd.read_csv("computational_benchmark_stats.csv")])

df_benchmark.to_csv("computational_benchmark_full.csv", index=False)

summary = (
    df_benchmark
    .groupby(["model_family","model","horizon_label"])
    .agg(
        mean_time=("training_time_sec","mean"),
        mean_time_per_series=("time_per_series_sec","mean")
    )
    .reset_index()
)

summary.to_csv("computational_benchmark_summary.csv", index=False)

summary.sort_values(['horizon_label', "mean_time"])

,model_family,model,horizon_label,mean_time,mean_time_per_series
44,stats,Naive,1 days,0.030523,0.010174
48,stats,SeasonalNaive-56,1 days,0.031315,0.010438
52,stats,SeasonalNaive-8,1 days,0.067749,0.022583
8,ml,XGBoost,1 days,20.544613,0.622564
28,neural,NBEATS-I,1 days,28.236438,0.855650
0,ml,LightGBM,1 days,29.340147,0.889095
32,neural,NHITS,1 days,29.929890,0.906966
36,stats,Arima,1 days,31.814013,10.604671
24,neural,NBEATS-G,1 days,35.130385,1.064557
40,stats,ETS,1 days,69.272963,23.090988


<!-- ### 1) Universo experimental e unidade de análise

A análise considera dados de reanálise do **Copernicus Atmosphere Monitoring Service (CAMS)** em grade regular, adequados para avaliação metodológica de previsão espacial consistente [1].

* Região de estudo: células localizadas nos estados
  RS, SC, PR, SP, MS, MG, RJ e ES.
* Unidade de previsão: **célula espacial** (latitude × longitude).
* Cada célula é tratada como **uma série temporal independente**, conforme prática comum em modelos globais de forecasting [2].

---

### 2) Resolução temporal e agregação

Como o objetivo envolve horizontes de **1 a 30 dias**, os dados horários são agregados para **resolução diária**:

* `ds`: data (agregação diária do `valid_time`)
* `y`: média diária da concentração do poluente

Essa escolha reduz ruído de alta frequência e melhora a estabilidade de modelos globais [3].

---

### 3) Construção dos datasets univariados

Para cada poluente
$$ p \in {\text{PM2.5, PM10, O}_3, \text{NO}_2}, $$
é construído um dataset no formato *long* exigido pela Nixtla [4]:

* `unique_id`: identificador da célula espacial
* `ds`: tempo (diário)
* `y`: concentração do poluente (p)

Cada poluente é tratado como **experimento independente**, evitando contaminação multivariada nesta RQ [5].

---

### 4) Normalização por série

Para viabilizar o treino de **modelos globais multi-séries**, cada série é normalizada **individualmente**, com parâmetros estimados **exclusivamente no conjunto de treino** [6].

Essa etapa é necessária devido à heterogeneidade de escala entre células (urbanas, rurais, industriais), conforme discutido na literatura de modelos globais [7].

---

### 5) Split temporal (anti-vazamento)

O particionamento é feito **estritamente no tempo**, comum a todos os modelos e poluentes:

* **Treino**: período inicial
* **Validação**: período intermediário (early stopping dos modelos neurais)
* **Teste**: período final (avaliação)

Nenhuma informação do conjunto de teste é utilizada em normalização, definição de percentis ou ajuste de hiperparâmetros [8].

---

### 6) Horizontes de previsão e janela de entrada

Os modelos são avaliados separadamente para os horizontes:
$$ H \in {1, 3, 7, 14, 30} \text{ dias} $$

* Janela de entrada (`input_size`) fixa para todos os modelos
* Um modelo é treinado por horizonte (direct forecasting), evitando propagação de erro recursiva [9].

---

### 7) Modelos avaliados

#### 7.1 Baselines ingênuos (locais)

* Persistência (Naive)
* Sazonal ingênuo (lag semanal)

Esses modelos fornecem referências mínimas e são amplamente utilizados como baseline em forecasting ambiental [10].

---

#### 7.2 Modelos estatísticos clássicos (locais)

* **AutoARIMA**
* **AutoETS**

Cada modelo estatístico é ajustado **individualmente por célula**, respeitando sua formulação tradicional [11].

---

#### 7.3 Baseline moderno explicável (global)

* **LightGBM global**, via MLForecast
* Uso de lags temporais e variáveis de calendário

Modelos globais baseados em árvores são reconhecidos como baselines fortes e interpretáveis em benchmarking moderno [12].

---

#### 7.4 Modelos neurais avaliados (globais)

* **N-BEATS** [13]
* **N-HiTS** [14]

Ambos são treinados como **modelos globais univariados**, compartilhando parâmetros entre todas as séries, prática estabelecida na literatura de deep learning para séries temporais [2].

---

### 8) Treinamento e hiperparâmetros

Para garantir comparabilidade e reprodutibilidade:

* Hiperparâmetros majoritariamente **off-the-shelf**
* Ajuste preliminar limitado a um cenário piloto
* Parâmetros **congelados** para todos os poluentes e horizontes

Essa estratégia segue recomendações da literatura comparativa, priorizando robustez em detrimento de tuning agressivo [7,15].

---

### 9) Métricas de avaliação

#### 9.1 Desempenho médio

* MAE
* RMSE
* sMAPE (avaliado com cautela para valores próximos de zero)

---

#### 9.2 Avaliação orientada a extremos

Para cada célula e poluente, os limiares **p95 e p99** são definidos no conjunto de treino.

No conjunto de teste, avalia-se:

* **MAE condicional** acima de p95 e p99
* **Detecção de eventos extremos**:

  * Precision
  * Recall
  * F1-score

Essa abordagem é recomendada em estudos ambientais, onde eventos raros são os mais relevantes do ponto de vista sanitário [16].

---

### 10) Agregação espacial e comparação estatística

Para cada (poluente, horizonte, modelo):

* Métricas são calculadas **por célula**
* Resultados agregados como **mediana e intervalo interquartil (p25–p75)**

Comparações entre modelos locais e globais são feitas por **teste pareado de Wilcoxon signed-rank**, avaliando se as distribuições de erro diferem significativamente [7].

---

### 11) Critério de desempenho competitivo (RQ1)

N-BEATS e N-HiTS são considerados **competitivos** quando:

1. Superam a persistência (skill score positivo)
2. Apresentam desempenho próximo ou superior ao LightGBM global
3. Mantêm desempenho estável em eventos extremos (p95/p99)

Esse critério reflete práticas consolidadas em estudos comparativos de forecasting [7,12].

--- -->

## Referências

* Buonanno A, Caliano M, Pontecorvo A, Sforza G, Valenti M, Graditi G. Global vs. Local Models for Short-Term Electricity Demand Prediction in a Residential/Lodging Scenario. Energies. 2022 Mar 10;15(6):2037. Available from: https://doi.org/10.3390/en15062037
1. Inness A, et al. *The CAMS reanalysis of atmospheric composition*. Atmos Chem Phys. 2019.
2. Salinas D, et al. *DeepAR: Probabilistic forecasting with autoregressive recurrent networks*. Int J Forecast. 2020.
3. Hyndman RJ, Athanasopoulos G. *Forecasting: Principles and Practice*. OTexts; 2021.
4. Nixtla. *NeuralForecast documentation*. 2023.
5. Box GEP, Jenkins GM. *Time Series Analysis: Forecasting and Control*. Holden-Day; 1976.
6. Makridakis S, et al. *Statistical and Machine Learning forecasting methods*. PLoS One. 2018.
7. Bianchi FM, et al. *Local vs global models for energy forecasting*. Energies. 2022.
8. Bergmeir C, Benítez JM. *On the use of cross-validation for time series predictor evaluation*. Inf Sci. 2012.
9. Taieb SB, Hyndman RJ. *Recursive and direct multi-step forecasting*. Int J Forecast. 2014.
10. Hyndman RJ. *Benchmarks for time series forecasting*. Int J Forecast. 2006.
11. Hyndman RJ, Khandakar Y. *Automatic time series forecasting*. J Stat Softw. 2008.
12. Makridakis S, et al. *The M5 accuracy competition*. Int J Forecast. 2022.
13. Oreshkin BN, et al. *N-BEATS: Neural basis expansion analysis for time series forecasting*. ICLR. 2020.
14. Challu C, et al. *N-HiTS: Neural hierarchical interpolation for time series forecasting*. AAAI. 2023.
15. Montero-Manso P, Hyndman RJ. *Principles and algorithms for forecasting groups of time series*. Int J Forecast. 2021.
16. Dabass A, et al. *Extreme air pollution events and health impacts*. Environ Health Perspect. 2018.